Okay I still should check that boltzmann stuff is working properly before doing the Q-values, so try running the boltzmann on the reward matrix rather than the q values and check that it gives you the distribution you'd expect just to make sure I'm not messing anything up there. 

Then if that's all fine, which it probably is, have another look at the Q-value and V-Value calculations in learn, and if they look right then step through and check that it's doing what I expect them to be doing. 

If it's something wrong with one of those then that might fix the learning side of things as well, but at least we'll have a proper trajectory to learn from because atm the one it's creating, and the resulting policy is abysmal, so fix that first, get the expert to perform more expertly and then you can check to see if there are problems with the learning algorithm :) 

Issue has to be in policy iteration somewhere 

In [1]:
%reload_ext autoreload
from env import SingleStateSpace 
from generate_trajectory import generate_trajectory
from policy import Boltzmann 
from learn import compute_q_with_values
from RewardFunctions import SingleStateReward
import numpy as np 
from learn import policy_iteration
from utils import normalise_pi
from policy_walk import policy_walk 
import matplotlib.pyplot as plt 

In [2]:

trajectory_length = 20 
#set up the agent and their policy 
env = SingleStateSpace(n_actions = 10, discount_rate = 0.1, R_max = 5)
#so we have access to env.reward
observation_times = np.cumsum(np.random.uniform(0, 2, size=trajectory_length))
print("Generated observation times:") 
print(observation_times)

# Use meshgrid to create a grid of all possible combinations of s, a, and t
s_grid, a_grid, t_grid = np.meshgrid(env.states, env.actions, observation_times, indexing='ij')
R = np.vectorize(env.reward)(s_grid,a_grid,t_grid)

# Calculate the reward for each combination using vectorized operations - oh I'm not actually using a boltzmann policy here I'm just using a normalised reward vector? 
(optimal_pi, optimal_values) = policy_iteration(env, len(observation_times), R)
optimal_q_values = np.ones((env.n_states, env.n_actions, len(observation_times)))
for s in range(env.n_states):
    for a in range(env.n_actions): 
        for t in range(len(observation_times)):
            optimal_q_values[s,a,t] = compute_q_with_values(env,s,a,t,optimal_values,R) 

b = Boltzmann(optimal_q_values, env.actions, alpha = 1.5)
observations = generate_trajectory(env,b,observation_times)

print("Generated trajectory") 
print(observations)

n_samples = 100 
learned_rewards = [] 
for i in range(n_samples): 
    if i%10 == 0: 
        print(".")
    learned_rewards.append((policy_walk(env,observations,optimal_q_values)))

# print("Running policy walk") 
# learned_R = policy_walk(env, observations, optimal_q_values) # there's a better way than passing in the Q* values but I cba dealing with it now 
# print("finished learning policy")

Generated observation times:
[ 0.70071022  1.94935825  2.60494554  3.24491636  3.24983847  3.87262539
  3.98477175  5.65226309  6.70209691  8.29156639 10.10643484 11.79140937
 11.80764542 12.65219259 12.91667317 14.08660098 15.13130241 15.4880177
 16.17398077 16.2282645 ]
Generated trajectory
[[0, 1, 0], [0, 5, 1], [0, 1, 2], [0, 7, 3], [0, 3, 4], [0, 2, 5], [0, 4, 6], [0, 5, 7], [0, 3, 8], [0, 8, 9], [0, 9, 10], [0, 7, 11], [0, 8, 12], [0, 2, 13], [0, 9, 14], [0, 1, 15], [0, 5, 16], [0, 4, 17], [0, 3, 18], [0, 4, 19]]
.


KeyboardInterrupt: 

In [ ]:
print(learned_R.shape)
for t in range(len(observations)):
    plt.plot(learned_R[0,:,t])
    plt.show()

In [ ]:

b = Boltzmann(optimal_q_values, env.actions)

#What is my initial policy? Everywhere I've seen, boltzmann is defined using the Q-values but you
#need a policy to find the Q-values so what is the actual policy? 
#Is it just like an exponential style distribution over the rewards? 
# observations = generate_trajectory(env, boltzmann_policy, observation_times)

alphas = [0.001, 0.01, 0.1, 0.5, 1, 1.5, 2, 4, 8, 16, 32, 64]
for alpha in alphas: 
    b = Boltzmann(optimal_q_values,env.actions,alpha) 
    
    count = np.zeros(env.n_actions)
    for i in range(100): 
        count[b(0,0)] += 1
    plt.plot(count) 
    asd = "alpha = " + format(alpha)
    plt.title(asd)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt 
for t in range(len(observations)):
    plt.plot(R[0,:,t])
    plt.show()